In [87]:
import pandas as pd
import numpy as np

In [88]:
df = pd.read_csv('https://pfabankapi.app.cloud.gov/api/failures?fields=NAME%2CCERT%2CFIN%2CCITYST%2CFAILDATE%2CSAVR%2CRESTYPE%2CCOST%2CRESTYPE1%2CCHCLASS1%2CQBFDEP%2CQBFASSET&filters=FAILYR%3A%5B1934%20TO%202023%5D&limit=10000&react=true&sort_by=FAILDATE&sort_order=desc&subtotal_by=RESTYPE&total_fields=QBFDEP%2CQBFASSET%2CCOST&format=csv&download=true&filename=bank-data')
df

,CERT,CHCLASS1,CITYST,COST,FAILDATE,FIN,ID,NAME,QBFASSET,QBFDEP,RESTYPE,RESTYPE1,SAVR
0,8758.0,NM,"SAC CITY, IA",14804.0,11/3/2023,10545,4109,CITIZENS BANK,60448.0,52311.0,FAILURE,PA,DIF
1,25851.0,SM,"ELKHART, KS",54167.0,7/28/2023,10544,4108,HEARTLAND TRI-STATE BANK,139446.0,130110.0,FAILURE,PA,DIF
2,59017.0,NM,"SAN FRANCISCO, CA",16566779.0,5/1/2023,10543,4107,FIRST REPUBLIC BANK,212638872.0,176436706.0,FAILURE,PA,DIF
3,57053.0,NM,"NEW YORK, NY",2787270.0,3/12/2023,10540,4106,SIGNATURE BANK,110363650.0,88612911.0,FAILURE,PA,DIF
4,24735.0,SM,"SANTA CLARA, CA",19671712.0,3/7/2023,10539,4105,SILICON VALLEY BANK,209026000.0,175378000.0,FAILURE,PA,DIF
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4104,NaN,NM,"LEWISPORT, KY",NaN,8/6/1934,0,5,BANK OF LEWISPORT,81.0,68.0,FAILURE,PO,FDIC
4105,NaN,N,"LIMA, MT",NaN,7/18/1934,0,3,FIRST NATIONAL BANK OF LIMA,91.0,42.0,FAILURE,PO,FDIC
4106,NaN,NM,"FLORENCE, IN",NaN,7/18/1934,0,4,FLORENCE DEPOSIT BANK,105.0,69.0,FAILURE,PO,FDIC
4107,NaN,NM,"EAST PEORIA, IL",NaN,5/28/1934,0,1,FON DU LAC STATE BANK,374.0,238.0,FAILURE,PO,FDIC


In [89]:
# Question 1 : According to our document, how many bank failures have there been since the FDIC was opened?
filt = df['RESTYPE'] == 'FAILURE'
df = df.loc[filt]
df['RESTYPE'].value_counts().sum()

3516

In [90]:
# Question 2 : What was the earliest failure in our data set? What is the most recent failure in our data set?
df['FAILDATE'] = pd.to_datetime(df['FAILDATE'])
# df.sort_values(by='FAILDATE').head(1) # (Oldest)
Newst = df.sort_values(by='FAILDATE',ascending=False).head(1).index # (Newest)
Oldest = df.sort_values(by='FAILDATE',ascending=True).head(1).index # (Oldest)
df.loc[[Oldest[0],Newst[0]]]

C:\Users\albrr\AppData\Local\Temp\ipykernel_3292\2207272439.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['FAILDATE'] = pd.to_datetime(df['FAILDATE'])


,CERT,CHCLASS1,CITYST,COST,FAILDATE,FIN,ID,NAME,QBFASSET,QBFDEP,RESTYPE,RESTYPE1,SAVR
4108,NaN,NM,"PITTSBURGH, PA",NaN,1934-04-19,0,2,BANK OF AMERICA TRUST CO.,1435.0,1064.0,FAILURE,PO,FDIC
0,8758.0,NM,"SAC CITY, IA",14804.0,2023-11-03,10545,4109,CITIZENS BANK,60448.0,52311.0,FAILURE,PA,DIF


In [91]:
df = df.set_index('FAILDATE')

In [92]:
# Question 3 : In which five years did the greatest number of banks fail?
# after solved notes... i interperted the question wrong he wanted the 5 years that had the biggies falier and to group by 5 and get the biggies one 
df['RESTYPE'].resample('5YE').count().sort_values(ascending=False).head(1)

FAILDATE
1989-12-31    1280
Name: RESTYPE, dtype: int64

In [93]:
# proof that the above smpling is accurate.
year5start = 1984
year5End = 1989
df = df.reset_index()
filt = ( df["FAILDATE"] > pd.to_datetime(f'{year5start}-12-31') ) & (df["FAILDATE"] < pd.to_datetime(f'{year5End}-12-31')) 
numRows = df.loc[filt].shape[0]

print(f'the number of row between the 5 year period of {year5start} and {year5End} is {numRows} rows')

the number of row between the 5 year period of 1984 and 1989 is 1280 rows


In [94]:
# Question 4 : In which three states were the greatest number of failed banks?
df['CITYST'].apply(lambda x:x[x.find(',')+2:]).value_counts().sort_values(ascending=False).head(3)

CITYST
TX    737
CA    238
IL    183
Name: count, dtype: int64

In [95]:
# Question 5 : What was the average market capitalization of the banks that failed? Given a capitalization of $200b, did that make SVB above, below, or about average?
avCap = df['QBFASSET'].mean()
print(f'the average captilaization of failded banks is {avCap} million. which is far less the 200 billion cap that SVB had ')

the average captilaization of failded banks is 568715.9765230312 million. which is far less the 200 billion cap that SVB had 


In [96]:
# Question 6 : When was the most recent failure greater than SVB?
filt = df['FAILDATE'] > df.loc[4,'FAILDATE'] 
df.loc[filt].loc[2]

FAILDATE    2023-05-01 00:00:00
CERT                    59017.0
CHCLASS1                     NM
CITYST        SAN FRANCISCO, CA
COST                 16566779.0
FIN                       10543
ID                         4107
NAME        FIRST REPUBLIC BANK
QBFASSET            212638872.0
QBFDEP              176436706.0
RESTYPE                 FAILURE
RESTYPE1                     PA
SAVR                        DIF
Name: 2, dtype: object

In [102]:
# Question 7 : Bank failures can be resolved in several different ways. How often, historically, have we seen each resolution? Were the odds good that SVB's uninsured depositors would get their money?
print(df['RESTYPE1'].value_counts()) # refer to the yaml file to understand the codes by searching the file for (RESTYPE1)

RESTYPE1
PA      2102
PO       573
IDT      386
P&A      251
PI       140
MGR       37
ABT       16
DINB       9
REP        2
Name: count, dtype: int64


In [98]:
# Question 8 : What about bank failures in the last 25 years -- if we just look at those, do the odds change?
filt = df['FAILDATE'] > pd.to_datetime('01-01-2000') 
df.loc[filt,'RESTYPE1'].value_counts()

RESTYPE1
PA      510
PI       32
PO       23
DINB      8
Name: count, dtype: int64

In [103]:
# Question 9 : What was the mean estimated loss in bank failures? What proportion of a bank's assets did this generally involve?
print(df['COST'].mean())

82049.00374914792
